# Automated ML

In the cell below, we import all the dependencies that we need to complete the project.

In [18]:
from azureml.core import Workspace, Dataset
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.pipeline.steps import AutoMLStep

In [19]:
from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0]+ ' - ')
        elif hasattr(step[1], '_base_learners') and hasattr(step[1], '_meta_learner'):
            print("\nMeta Learner")
            pprint(step[1]._meta_learner)
            print()
            for estimator in step[1]._base_learners:
                print_model(estimator[1], estimator[0]+ ' - ')
        else:
            pprint(step[1].get_params())
            print()

## Dataset

### Overview

For this final project we'll be using the [Heart Failure Prediction](https://www.kaggle.com/datasets/andrewmvd/heart-failure-clinical-data) dataset from Kaggle.

This dataset contains medical records for 299 patients with hear failure,
along with a column indicating survival as a binary variable.
Our goal is to predict survival from the rest of the data.
This means that we are faced with a classification problem with two classes.

In [20]:
workspace = Workspace.from_config()

experiment_name = 'edu_hf_automl_exp'
experiment = Experiment(workspace, experiment_name)

compute_cluster_name = "edu-compute-cluster"
compute_target = workspace.compute_targets[compute_cluster_name]

dataset_name = 'edu_heart_failure_dataset'
dataset = Dataset.get_by_name(workspace, name=dataset_name)

In [21]:
df = dataset.to_pandas_dataframe()
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.00,0,582,0,20,1,265000.00,1.90,130,1,0,4,1
1,55.00,0,7861,0,38,0,263358.03,1.10,136,1,0,6,1
2,65.00,0,146,0,20,0,162000.00,1.30,129,1,1,7,1
3,50.00,1,111,0,20,0,210000.00,1.90,137,1,0,7,1
4,65.00,1,160,1,20,0,327000.00,2.70,116,0,0,8,1


## AutoML Configuration

In the cells below we use the AutoMLConfig class to configure several aspects of AutoML:
- task is set to classification, which matches our use case
- training_data is set to the dataset we instantiated above
- label_column_name is the name of the target column, DEATH_EVENT
- n_cross_validations=5 splits the dataset into five folds, using each of them in sequence as a test set while training on the remaining four, to better assess model performance
- as primary metric to evaluate experiments we choose AUC weighted
- early stopping is enabled, so that the experiment can end early if results are discouraging
- we set a timeout of one hour to avoid running out of time with the Udacity VM
- as compute_target we choose a compute cluster we created beforehand

In [22]:
target_column = "DEATH_EVENT"

automl_settings = {
    "task": "classification",
    "primary_metric": "AUC_weighted",
    "training_data": dataset,
    "label_column_name": target_column,
    "n_cross_validations": 5,
    "compute_target": compute_target,
    "enable_early_stopping": True,
    "experiment_timeout_hours": 1
}

automl_config = AutoMLConfig(**automl_settings)

In [23]:
automl_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
edu_hf_automl_exp,AutoML_5e6952ca-0ced-4c1c-ba88-5369f9fae997,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

In [24]:
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

In the cells below, we get the best model from the AutoML experiments and display all the properties of the model.



In [25]:
best_run, fitted_model = automl_run.get_output()

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

In [26]:
print(f"Best run id: {best_run.id}")

Best run id: AutoML_5e6952ca-0ced-4c1c-ba88-5369f9fae997_43


In [27]:
print(best_run.get_metrics())

{'precision_score_micro': 0.8528248587570622, 'average_precision_score_micro': 0.9245017105368405, 'matthews_correlation': 0.6652989706269239, 'accuracy': 0.8528248587570622, 'f1_score_weighted': 0.849003302667834, 'average_precision_score_weighted': 0.930655835483209, 'precision_score_weighted': 0.8643369099065332, 'recall_score_macro': 0.8241285991140643, 'average_precision_score_macro': 0.9084979916884868, 'AUC_weighted': 0.9220907622739019, 'AUC_micro': 0.9218927351655015, 'f1_score_micro': 0.8528248587570622, 'balanced_accuracy': 0.8241285991140643, 'norm_macro_recall': 0.6482571982281284, 'log_loss': 0.3828329747370618, 'weighted_accuracy': 0.8716568485254653, 'recall_score_weighted': 0.8528248587570622, 'AUC_macro': 0.9220907622739019, 'f1_score_macro': 0.8227216899602297, 'recall_score_micro': 0.8528248587570622, 'precision_score_macro': 0.8431721603437607, 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_5e6952ca-0ced-4c1c-ba88-5369f9fae997_43/confusion_matrix',

In [29]:
print_model(fitted_model)

datatransformer
{'enable_dnn': False,
 'enable_feature_sweeping': True,
 'feature_sweeping_config': {},
 'feature_sweeping_timeout': 86400,
 'featurization_config': None,
 'force_text_dnn': False,
 'is_cross_validation': True,
 'is_onnx_compatible': False,
 'observer': None,
 'task': 'classification',
 'working_dir': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook239639/code/Users/odl_user_239639/nd00333-capstone-master'}

prefittedsoftvotingclassifier
{'estimators': ['32', '33', '26', '20', '38', '30', '39', '21', '6'],
 'weights': [0.14285714285714285,
             0.14285714285714285,
             0.14285714285714285,
             0.07142857142857142,
             0.07142857142857142,
             0.07142857142857142,
             0.07142857142857142,
             0.14285714285714285,
             0.14285714285714285]}

32 - standardscalerwrapper
{'class_name': 'StandardScaler',
 'copy': True,
 'module_name': 'sklearn.preprocessing._data',
 'with_mean': False,
 'with_std':

## Model Deployment

We do not deploy this model, but we register it nevertheless.

In [30]:
model_name = best_run.properties['model_name']
model = best_run.register_model(model_name=model_name, model_path="outputs")

In [31]:
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
